# Setup

In [14]:
from pathlib import Path
import sys
import os

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import Isomap
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from hassediagram.hasse_diagram import hasse_graphviz


sys.path.append(str(Path(os.getcwd()).parent))
from mcda.dataset import MCDADataset
from mcda.uta import Criterion, calculate_uta_gms
from methods.autoencoder import DominanceAutoEncoder
from methods.mvu import MaximumVarianceUnfolding

In [15]:
def get_methods(n: int) -> dict:
    """
    Get a dictionary of methods for dimensionality reduction.

    Parameters
    ----------
    n (int): Number of components for the methods.

    Returns
    -------
    dict: Dictionary of methods.

    """
    return {
        "PCA": Pipeline([("scaler", StandardScaler()), ("pca", PCA(n_components=n, random_state=42))]),
        "KernelPCA": Pipeline([("scaler", StandardScaler()), ("kpca", KernelPCA(n_components=n, random_state=42))]),
        "Isomap": Pipeline([("scaler", StandardScaler()), ("isomap", Isomap(n_components=n))]),
        "MVU": Pipeline([("scaler", StandardScaler()), ("mvu", MaximumVarianceUnfolding(n_components=n, seed=42))]),
        "DAE": Pipeline(
            [
                ("scaler", StandardScaler()),
                ("dae", DominanceAutoEncoder(latent_dim=n, num_epochs=1000, random_state=42, verbose=False)),
            ]
        ),
    }

# Preferences

In [35]:
preferences = {
    1: [[('a3', 'a9')], [('a8', 'a11')], [('a6', 'a12')], [('a6', 'a11')], [('a6', 'a2')], [('a7', 'a8')], [('a1', 'a2')], [('a5', 'a8')], [('a3', 'a2')], [('a1', 'a9')]],
    2: [[('a3', 'a9'), ('a1', 'a9')], [('a8', 'a11'), ('a4', 'a2')], [('a6', 'a12'), ('a5', 'a8')], [('a6', 'a11'), ('a1', 'a4')], [('a6', 'a2'), ('a10', 'a5')], [('a7', 'a8'), ('a1', 'a11')], [('a1', 'a2'), ('a1', 'a7')], [('a5', 'a8'), ('a12', 'a8')], [('a3', 'a2'), ('a6', 'a4')], [('a1', 'a9'), ('a3', 'a4')]],
    3: [[('a3', 'a9'), ('a1', 'a9'), ('a1', 'a11')], [('a8', 'a11'), ('a4', 'a2'), ('a3', 'a9')], [('a6', 'a12'), ('a5', 'a8'), ('a3', 'a9')], [('a6', 'a11'), ('a1', 'a4'), ('a6', 'a12')], [('a6', 'a2'), ('a10', 'a5'), ('a7', 'a12')], [('a7', 'a8'), ('a1', 'a11'), ('a1', 'a9')], [('a1', 'a2'), ('a1', 'a7'), ('a6', 'a9')], [('a5', 'a8'), ('a12', 'a8'), ('a6', 'a8')], [('a3', 'a2'), ('a6', 'a4'), ('a4', 'a9')], [('a1', 'a9'), ('a3', 'a4'), ('a10', 'a2')]],
    4: [[('a3', 'a9'), ('a1', 'a9'), ('a1', 'a11'), ('a1', 'a8')], [('a8', 'a11'), ('a4', 'a2'), ('a3', 'a9'), ('a6', 'a11')], [('a6', 'a12'), ('a5', 'a8'), ('a3', 'a9'), ('a4', 'a11')], [('a6', 'a11'), ('a1', 'a4'), ('a6', 'a12'), ('a10', 'a3')], [('a6', 'a2'), ('a10', 'a5'), ('a7', 'a12'), ('a5', 'a7')], [('a7', 'a8'), ('a1', 'a11'), ('a1', 'a9'), ('a12', 'a11')], [('a1', 'a2'), ('a1', 'a7'), ('a6', 'a9'), ('a1', 'a6')], [('a5', 'a8'), ('a12', 'a8'), ('a6', 'a8'), ('a6', 'a3')], [('a3', 'a2'), ('a6', 'a4'), ('a4', 'a9'), ('a4', 'a11')], [('a1', 'a9'), ('a3', 'a4'), ('a10', 'a2'), ('a12', 'a2')]],
    5: [[('a3', 'a9'), ('a1', 'a9'), ('a1', 'a11'), ('a1', 'a8'), ('a7', 'a3')], [('a8', 'a11'), ('a4', 'a2'), ('a3', 'a9'), ('a6', 'a11'), ('a7', 'a9')], [('a6', 'a12'), ('a5', 'a8'), ('a3', 'a9'), ('a4', 'a11'), ('a1', 'a7')], [('a6', 'a11'), ('a1', 'a4'), ('a6', 'a12'), ('a10', 'a3'), ('a10', 'a2')], [('a6', 'a2'), ('a10', 'a5'), ('a7', 'a12'), ('a5', 'a7'), ('a10', 'a3')], [('a7', 'a8'), ('a1', 'a11'), ('a1', 'a9'), ('a12', 'a11'), ('a5', 'a12')], [('a1', 'a2'), ('a1', 'a7'), ('a6', 'a9'), ('a1', 'a6'), ('a6', 'a12')], [('a5', 'a8'), ('a12', 'a8'), ('a6', 'a8'), ('a6', 'a3'), ('a4', 'a9')], [('a3', 'a2'), ('a6', 'a4'), ('a4', 'a9'), ('a4', 'a11'), ('a1', 'a7')], [('a1', 'a9'), ('a3', 'a4'), ('a10', 'a2'), ('a12', 'a2'), ('a4', 'a9')]],
    6: [[('a3', 'a9'), ('a1', 'a9'), ('a1', 'a11'), ('a1', 'a8'), ('a7', 'a3'), ('a4', 'a2')], [('a8', 'a11'), ('a4', 'a2'), ('a3', 'a9'), ('a6', 'a11'), ('a7', 'a9'), ('a1', 'a9')], [('a6', 'a12'), ('a5', 'a8'), ('a3', 'a9'), ('a4', 'a11'), ('a1', 'a7'), ('a7', 'a11')], [('a6', 'a11'), ('a1', 'a4'), ('a6', 'a12'), ('a10', 'a3'), ('a10', 'a2'), ('a4', 'a9')], [('a6', 'a2'), ('a10', 'a5'), ('a7', 'a12'), ('a5', 'a7'), ('a10', 'a3'), ('a3', 'a8')], [('a7', 'a8'), ('a1', 'a11'), ('a1', 'a9'), ('a12', 'a11'), ('a5', 'a12'), ('a1', 'a5')], [('a1', 'a2'), ('a1', 'a7'), ('a6', 'a9'), ('a1', 'a6'), ('a6', 'a12'), ('a1', 'a5')], [('a5', 'a8'), ('a12', 'a8'), ('a6', 'a8'), ('a6', 'a3'), ('a4', 'a9'), ('a5', 'a4')], [('a3', 'a2'), ('a6', 'a4'), ('a4', 'a9'), ('a4', 'a11'), ('a1', 'a7'), ('a3', 'a8')], [('a1', 'a9'), ('a3', 'a4'), ('a10', 'a2'), ('a12', 'a2'), ('a4', 'a9'), ('a4', 'a2')]],
}

# Dataset

In [36]:
dataset: MCDADataset = MCDADataset.read_csv("../data/insulating-materials/dataset.csv")

# Generating diagrams

In [45]:
for preference in [l[0] for l in preferences.values()]:
    number_of_preference = len(preference)
    for n in [2, 3, 4, 5, 6]:
        methods = get_methods(n)
        for method_name, method in methods.items():
            transformed_data = method.fit_transform(dataset.data)
            df_m = (
                pd.DataFrame(transformed_data, index=dataset.data.index, columns=range(n)).map(lambda x: f"{x:.4f}").astype(np.float64)
            )
            criteria = [Criterion(name, points=n_points) for name in df_m.columns]
            df_utagms = calculate_uta_gms(df_m, preference, criteria)
            dot = hasse_graphviz(df_utagms.values)
            dot.graph_attr.update(dpi='300')
            dot.render(filename=method_name, directory=f"../data/insulating-materials/output/preferences_{number_of_preference}/diagrams/dim_{n}", format="png", cleanup=True)